In [2]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
 
def xavier_init(fan_in, fan_out, constant = 1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out),
                             minval=low, maxval=high,
                             dtype=tf.float32)
 
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self, n_input, n_hidden, transfer_function=tf.nn.softplus,
                 optimizer=tf.train.AdamOptimizer(), scale=0.1):
        self.n_input = n_input           # 输入变量数
        self.n_hidden = n_hidden         # 隐含层节点数
        self.transfer = transfer_function  # 隐含层激活函数
        self.scale = tf.placeholder(tf.float32)  #高斯噪声稀疏
        self.training_scale = scale
        network_weights = self._initialize_weights()   # 我们只用了一个隐含层
        self.weights = network_weights
 
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(
            self.x + scale * tf.random_normal((n_input, )),
            self.weights['w1']), self.weights['b1']
        ))
        self.reconstruction = tf.add(tf.matmul(self.hidden,
                                     self.weights['w2']),
                                     self.weights['b2'])
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(
            self.reconstruction, self.x), 2.0
        ))
        self.optimizer = optimizer.minimize(self.cost)
 
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
 
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype=tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden,
                                                  self.n_input], dtype=tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype=tf.float32))
        return all_weights
 
    # 计算损失cost及执行一步训练的函数partial_fit
    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer),
                                  feed_dict={self.x: X, self.scale: self.training_scale})
        return cost
 
    # 让session执行一个计算图节点self.cost，在测试会用到
    def calc_total_cost(self, X):
        return self.sess.run(self.cost,
                             feed_dict={self.x: X, self.scale: self.training_scale})
 
    # 返回自编码器隐含层的输出结果，目的是提供一个接口来获取抽象后的特征
    def transform(self, X):
        return self.sess.run(self.hidden, feed_dict={self.x: X,
                                                     self.scale: self.training_scale})
 
    # generate函数，将隐含层的输出结果作为输入
    # 通过之后的重建层将提取到的高阶特征复原为原始数据
    def generate(self, hidden = None):
        if hidden is None:
            hidden = np.random.normal(size=self.weights['b1'])
        return self.sess.run(self.reconstruction, feed_dict={self.hidden:hidden})
 
    # 整体运行一遍复原过程，包括提取高阶特征和通过高阶特征复原数据
    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict={self.x: X,
                                                              self.scale: self.training_scale})
 
    # 获取隐含层的权重 W1
    def getWeights(self):
        return self.sess.run(self.weights['w1'])
 
    # 获取隐含层的偏置系数 b1
    def getBiases(self):
        return self.sess.run(self.weights['b1'])
 
# read data to load mnist dataset
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
 
def standard_scale(X_train, X_test):
    # 标准化即让数据变成0均值，且标准差为1的分布
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test
 
# 定义一个获取随机block数据的函数，取一个从0到len(data)-batch_size之间的随机整数
# 这里是不放回抽样，可以提高数据的利益效率
def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data) - batch_size)
    return data[start_index: (start_index + batch_size)]
 
# 使用之前定义的standard_scale 函数对训练集，测试集进行标准化变换
X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)
 
n_samples = int(mnist.train.num_examples)  # 总训练样本数据
training_epochs = 20   # 最大训练的轮数
batch_size = 128   # 批次尺寸
display_step = 1
 
autoencoder = AdditiveGaussianNoiseAutoencoder(n_input=784,
                                               n_hidden=200,
                                               transfer_function=tf.nn.softplus,
                                               optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
                                               scale=0.01)
 
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(n_samples / batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train, batch_size)
 
        cost = autoencoder.partial_fit(batch_xs)
        avg_cost += cost / n_samples * batch_size
 
    if epoch % display_step == 0:
        print("Epoch:", '%04d' %(epoch + 1), 'cost=', "{:.9f}".format(avg_cost))
 
print("Total cost: " + str(autoencoder.calc_total_cost(X_test)))

Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Epoch: 0001 cost= 19778.613995455
Epoch: 0002 cost= 12368.050288636
Epoch: 0003 cost= 10632.428655682
Epoch: 0004 cost= 10459.903954545
Epoch: 0005 cost= 8816.775254545
Epoch: 0006 cost= 9416.449596591
Epoch: 0007 cost= 9161.720517614
Epoch: 0008 cost= 8512.037610227
Epoch: 0009 cost= 8642.015051705
Epoch: 0010 cost= 8196.783003409
Epoch: 0011 cost= 8319.180798864
Epoch: 0012 cost= 8933.030585795
Epoch: 0013 cost= 8721.528743182
Epoch: 0014 cost= 7570.707595455
Epoch: 0015 